In [132]:
import cv2
import numpy as np

In [133]:
# Load image
image_name = "test_original.jpg" # available images 1-4
log_steps = True        # if the different steps across the progress should be saved as image

image = cv2.imread(image_name)

In [134]:
# Find edges
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred = cv2.GaussianBlur(gray, (5, 5), 0)

if log_steps:
    cv2.imwrite('blurred.jpg', blurred)

edged = cv2.Canny(blurred, 75, 200)

if log_steps:
    cv2.imwrite('edged.jpg', edged)

In [135]:
# Close edges
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

closed = cv2.dilate(closed, None, iterations=2)
closed = cv2.erode(closed, None, iterations=2)

if log_steps:
    cv2.imwrite('closed.jpg', closed)

In [136]:
# Find contours
contours, _ = cv2.findContours(closed.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

if log_steps:
    contoured_image = image.copy()
    cv2.drawContours(contoured_image, contours[:100], -1, (0, 255, 0), 2)

    cv2.imwrite('contours.jpg', contoured_image)

In [137]:
def max_pairwise_distance(contour):
    hull = cv2.convexHull(contour, returnPoints=True)
    n = len(hull)
    if n < 2:
        return 0.0

    max_dist = 0
    j = 1
    for i in range(n):
        while True:
            d1 = np.linalg.norm(hull[i][0] - hull[j % n][0])
            d2 = np.linalg.norm(hull[i][0] - hull[(j + 1) % n][0])
            if d2 > d1:
                j += 1
            else:
                break
        max_dist = max(max_dist, np.linalg.norm(hull[i][0] - hull[j % n][0]))
    return max_dist

In [138]:
# Filter the contours

# Thresholds
min_contour_length = 600
min_area_size = 75
min_diameter = 400

filtered_contours = []

for cnt in contours:

    length = cv2.arcLength(cnt, True)
    area = cv2.contourArea(cnt)

    # Verify arclength of contour
    if length < min_contour_length or area <= min_area_size:
        continue

    # Verify diameter of contour
    diameter = max_pairwise_distance(cnt)
    if diameter <= min_diameter:
            continue

    # Passed all filters, keep it
    filtered_contours.append(cnt)

# Only keep the 25 most promising contours
filtered_contours = filtered_contours[:25]

if log_steps:
    filtered_edges = np.ones_like(image) * 255
    cv2.drawContours(filtered_edges, filtered_contours, -1, 255, thickness=cv2.FILLED)

    cv2.imwrite("filtered_edges.jpg", filtered_edges)

In [139]:
# Create an outer hull of the contours
concatenated_contour = np.vstack(filtered_contours)
hull = cv2.convexHull(concatenated_contour)

if log_steps:
    mask = np.zeros_like(image) + 150
    cv2.drawContours(mask, [hull], -1, (255, 255, 255), thickness=cv2.FILLED)

    cv2.imwrite("mask.jpg", mask)

In [140]:
# Simplify the hull shape
peri = cv2.arcLength(hull, True)
approx = cv2.approxPolyDP(hull, 0.015 * peri, True)

if log_steps:
    cornered = mask.copy()
    for corner in approx:
        cv2.circle(cornered, corner[0], 20, (0, 0, 255), 20)

    cv2.imwrite("cornered.jpg", cornered)

In [141]:
height, width, _ = image.shape

In [142]:
# Order points to allow perspective transform
def order_points(pts):
    rect = np.zeros((4, 2), dtype=np.float32)

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left

    return rect

In [143]:
approx.reshape((4, 2))

ValueError: cannot reshape array of size 10 into shape (4,2)

In [124]:
corners = order_points(approx.reshape((4, 2)))

In [125]:
sorted_points = order_points(np.array(corners))
(tl, tr, br, bl) = sorted_points

# Compute approximate width in pixels (horizontal sides)
width_a = np.linalg.norm(br - bl)
width_b = np.linalg.norm(tr - tl)
max_width = max(int(width_a), int(width_b))

# Compute approximate height in pixels (vertical sides)
height_a = np.linalg.norm(tr - br)
height_b = np.linalg.norm(tl - bl)
max_height = max(int(height_a), int(height_b))

In [126]:
print(sorted_points)

[[ 420.  161.]
 [ 692.  193.]
 [ 662. 1061.]
 [ 356. 1058.]]


In [127]:
output_width = 800

ratio = max_width / float(output_width)
output_height = int(max_height / ratio)

In [128]:
# Warp the image to only keep the receipt

src_pts = sorted_points
dst_pts = np.array([
    [0, 0],
    [output_width, 0],
    [output_width, output_height],
    [0, output_height]
], dtype=np.float32)

matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
result = cv2.warpPerspective(image.copy(), matrix, (output_width, output_height), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

if log_steps:
    cv2.imwrite("warped.jpg", result)

In [129]:
# Remove background and boost contrast
def local_contrast_filter(img):
    img = img.astype(np.float32)

    local_mean = cv2.blur(img, (15, 15))
    diff = img - local_mean

    adjusted = img + diff * 5

    adjusted_clipped = np.clip(adjusted, 0, 255)

    return adjusted_clipped.astype(np.uint8)

original_image = result.copy()
original_image_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

filtered = local_contrast_filter(original_image_gray)

if log_steps:
    cv2.imwrite("enhanced.jpg", filtered)

filtered[filtered > 50] = 255
res = filtered

In [130]:
# res = cv2.erode(res, None, iterations=1)
# res = cv2.dilate(res, None, iterations=1)

In [131]:
# Output final result
cv2.imwrite("final_result.jpg", res)

True